# Sharding and Parallel I/O

WebDataset datasets are usually split into many shards; this is both to achieve parallel I/O and to shuffle data.

Sets of shards can be given as a list of files, or they can be written using the brace notation, as in `openimages-train-{000000..000554}.tar`.

For example, the OpenImages dataset consists of 554 shards, each containing about 1 Gbyte of images. You can open the entire dataset as follows.

In [6]:
url = "http://storage.googleapis.com/nvdata-openimages/openimages-train-{000000..000554}.tar"
url = f"pipe:curl -L -s {url} || true"
dataset = (
    wds.WebDataset(url, shardshuffle=True)
    .shuffle(100)
    .decode("pil")
    .to_tuple("jpg;png", "json")
    .map_tuple(preproc)
    #.then(wds.map_tuple, preproc)
)
x, y = next(iter(dataset))
print(x.shape, str(y)[:50])

torch.Size([3, 224, 224]) [{'ImageID': 'a13866f13e7c1ea6', 'Source': 'active


Note the explicit use of both `shardshuffle=True` (for shuffling the shards) and the `.shuffle` processor (for shuffling samples inline).

When used with a standard Torch `DataLoader`, this will now perform parallel I/O and preprocessing.

The recommended way of using `IterableDataset` with `DataLoader` is to do the batching explicitly in the `Dataset`. You can also set a nominal length for a dataset.

In [7]:
url = "http://storage.googleapis.com/nvdata-openimages/openimages-train-{000000..000554}.tar"
url = f"pipe:curl -L -s {url} || true"
bs = 20

dataset = (
    wds.WebDataset(url, length=int(1e9) // bs)
    .shuffle(100)
    .decode("pil")
    .to_tuple("jpg;png", "json")
    .map_tuple(preproc, identity)
    .batched(20)
)

dataloader = torch.utils.data.DataLoader(dataset, num_workers=4, batch_size=None)
images, targets = next(iter(dataloader))
images.shape

torch.Size([20, 3, 224, 224])

The `ResizedDataset` is also helpful for connecting iterable datasets to `DataLoader`: it lets you set both a nominal and an actual epoch size; it will repeatedly iterate through the entire dataset and return data in chunks with the given epoch size.